## Gemini 기본 처리

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 5.5 MB/s eta 0:00:00


### GCP 환경설정 및 로그인

In [2]:
import os

PROJECT_ID = "ai-hangsik" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USE_VERTEX_AI = True #@param {type:"boolean"}
MODEL = "gemini-2.5-flash" #@param {type:"string"}

In [3]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project {PROJECT_ID}

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=tXfmy2O1Z34I5GJiIO4CTDTmX8YW1d&prompt=consent&token_usage=remote&access_type=offline&code_challenge=A_kN7E7VYbKIkhCeWCoILFbCdPzfGuvL89WRaaG80so&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJgC0mKyqnjghHS6rHtWnwSEy66-StTVL36w9dWLOlPd95U8_fCStmfu7h-8UVFkyg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

### Vertex AI Client 실행

In [4]:
import base64
from IPython.display import Image, display, Markdown

from google import genai
from google.genai import types
from google.genai.types import HttpOptions

client = genai.Client(
    vertexai=USE_VERTEX_AI,
    project=PROJECT_ID,
    location=REGION,)

In [17]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest", )->dict:
    """
    지정한 날짜에 두 통화 간의 환율을 조회합니다.

    Frankfurter API(https://api.frankfurter.app/)를 사용하여
    환율 데이터를 가져옵니다.

    인자:
        currency_from: 기준 통화(3자리 통화 코드). 기본값은 "USD"(미국 달러).
        currency_to: 대상 통화(3자리 통화 코드). 기본값은 "KRW"(원화).
        currency_date: 환율을 조회할 날짜. 기본값은 "latest"로 최신 환율 데이터를 의미합니다.
            과거 환율은 YYYY-MM-DD 형식으로 지정할 수 있습니다.

    반환값:
        dict: 환율 정보가 담긴 딕셔너리.
            예시: {"amount": 1.0, "base": "USD", "date": "2023-11-24", "rates": {"EUR": 0.95534}}
    """

    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )

    print(response.json())
    return response.json()


### Gemini Functaion calling

In [27]:
from google import genai
from google.genai.types import HttpOptions
from IPython.display import Image, display, Markdown

generate_content_config = types.GenerateContentConfig(
    system_instruction ="""당신을 사용자의 질문에 대해서 환율을 조회해 주는 어시스턴트입니다.
                           사용자가 환율을 조회할때는 get_exchange_rate 툴을 사용해주세요. """,

    tools = [get_exchange_rate]
)

response = client.models.generate_content(
      model = MODEL,
      contents =
        types.Content(
          role="user",
          parts=[
            types.Part.from_text(text="""현재 원(KRW) 달러(USD) 환율은 ? """)
          ]
      ),
      config = generate_content_config,
  )

print(response.text)



{'amount': 1.0, 'base': 'USD', 'date': '2025-07-23', 'rates': {'KRW': 1374.94}}
현재 1달러당 1374.94 원 입니다.


In [28]:
print(response)


sdk_http_response=HttpResponse(
  headers=<dict len=9>
) candidates=[Candidate(
  avg_logprobs=-0.035748925473954946,
  content=Content(
    parts=[
      Part(
        text='현재 1달러당 1374.94 원 입니다.'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>
)] create_time=datetime.datetime(2025, 7, 24, 4, 6, 22, 187140, tzinfo=TzInfo(UTC)) response_id='PrGBaIS2C6v5ld8Pn6CvkQ4' model_version='gemini-2.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=18,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=18
    ),
  ],
  prompt_token_count=359,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=359
    ),
  ],
  total_token_count=377,
  traffic_type=<TrafficType.ON_DEMAND: 'ON_DEMAND'>
) automatic_function_calling_history=[Content(
  parts=[
    Part(
      text='현재 원(KRW) 달러(USD) 

## End of Document